# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 11 2022 8:30AM,248309,10,LUP8000198592,"Lupin Research, Inc.",Released
1,Oct 11 2022 8:30AM,248311,10,7069346,"CLINUVEL, Inc.",Released
2,Oct 10 2022 4:28PM,248310,19,STR10112022-1,ACG North America LLC,Released
3,Oct 10 2022 4:09PM,248307,19,ACG-2102491526,ACG North America LLC,Released
4,Oct 10 2022 3:15PM,248300,10,0086065037,ISDIN Corporation,Executing
5,Oct 10 2022 3:15PM,248300,10,0086065045,ISDIN Corporation,Executing
6,Oct 10 2022 3:15PM,248300,10,0086065054,ISDIN Corporation,Executing
7,Oct 10 2022 3:15PM,248300,10,0086065080,ISDIN Corporation,Executing
8,Oct 10 2022 3:15PM,248300,10,0086065094,ISDIN Corporation,Executing
9,Oct 10 2022 3:15PM,248300,10,0086065098,ISDIN Corporation,Executing


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
12,248300,Released,44
13,248307,Released,1
14,248309,Released,1
15,248310,Released,1
16,248311,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
248300,16.0,44.0
248307,NaN,1.0
248309,NaN,1.0
248310,NaN,1.0
248311,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
248244,0.0,3.0
248260,26.0,1.0
248262,19.0,1.0
248278,0.0,1.0
248286,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
248244,0,3
248260,26,1
248262,19,1
248278,0,1
248286,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,248244,0,3
1,248260,26,1
2,248262,19,1
3,248278,0,1
4,248286,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,248244,,3
1,248260,26,1
2,248262,19,1
3,248278,,1
4,248286,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 11 2022 8:30AM,248309,10,"Lupin Research, Inc."
1,Oct 11 2022 8:30AM,248311,10,"CLINUVEL, Inc."
2,Oct 10 2022 4:28PM,248310,19,ACG North America LLC
3,Oct 10 2022 4:09PM,248307,19,ACG North America LLC
4,Oct 10 2022 3:15PM,248300,10,ISDIN Corporation
64,Oct 10 2022 3:03PM,248297,10,ISDIN Corporation
65,Oct 10 2022 2:57PM,248296,20,"ACI Healthcare USA, Inc."
66,Oct 10 2022 2:55PM,248290,16,"Emersa Waterbox, LLC"
67,Oct 10 2022 2:55PM,248288,18,"Emersa Waterbox, LLC"
68,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Oct 11 2022 8:30AM,248309,10,"Lupin Research, Inc.",,1
1,Oct 11 2022 8:30AM,248311,10,"CLINUVEL, Inc.",,1
2,Oct 10 2022 4:28PM,248310,19,ACG North America LLC,,1
3,Oct 10 2022 4:09PM,248307,19,ACG North America LLC,,1
4,Oct 10 2022 3:15PM,248300,10,ISDIN Corporation,16,44
5,Oct 10 2022 3:03PM,248297,10,ISDIN Corporation,,1
6,Oct 10 2022 2:57PM,248296,20,"ACI Healthcare USA, Inc.",,1
7,Oct 10 2022 2:55PM,248290,16,"Emersa Waterbox, LLC",,1
8,Oct 10 2022 2:55PM,248288,18,"Emersa Waterbox, LLC",,1
9,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 11 2022 8:30AM,248309,10,"Lupin Research, Inc.",1,
1,Oct 11 2022 8:30AM,248311,10,"CLINUVEL, Inc.",1,
2,Oct 10 2022 4:28PM,248310,19,ACG North America LLC,1,
3,Oct 10 2022 4:09PM,248307,19,ACG North America LLC,1,
4,Oct 10 2022 3:15PM,248300,10,ISDIN Corporation,44,16
5,Oct 10 2022 3:03PM,248297,10,ISDIN Corporation,1,
6,Oct 10 2022 2:57PM,248296,20,"ACI Healthcare USA, Inc.",1,
7,Oct 10 2022 2:55PM,248290,16,"Emersa Waterbox, LLC",1,
8,Oct 10 2022 2:55PM,248288,18,"Emersa Waterbox, LLC",1,
9,Oct 10 2022 1:29PM,248286,19,"GCH Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 11 2022 8:30AM,248309,10,"Lupin Research, Inc.",1,
1,Oct 11 2022 8:30AM,248311,10,"CLINUVEL, Inc.",1,
2,Oct 10 2022 4:28PM,248310,19,ACG North America LLC,1,
3,Oct 10 2022 4:09PM,248307,19,ACG North America LLC,1,
4,Oct 10 2022 3:15PM,248300,10,ISDIN Corporation,44,16


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Oct 11 2022 8:30AM,248309,10,"Lupin Research, Inc.",1,NaN
1,Oct 11 2022 8:30AM,248311,10,"CLINUVEL, Inc.",1,NaN
2,Oct 10 2022 4:28PM,248310,19,ACG North America LLC,1,NaN
3,Oct 10 2022 4:09PM,248307,19,ACG North America LLC,1,NaN
4,Oct 10 2022 3:15PM,248300,10,ISDIN Corporation,44,16.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Oct 11 2022 8:30AM,248309,10,"Lupin Research, Inc.",1,0.0
1,Oct 11 2022 8:30AM,248311,10,"CLINUVEL, Inc.",1,0.0
2,Oct 10 2022 4:28PM,248310,19,ACG North America LLC,1,0.0
3,Oct 10 2022 4:09PM,248307,19,ACG North America LLC,1,0.0
4,Oct 10 2022 3:15PM,248300,10,ISDIN Corporation,44,16.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1241461,50,16.0
15,744800,3,45.0
16,248290,1,0.0
18,248288,1,0.0
19,744903,3,0.0
20,248296,1,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1241461,50,16.0
1,15,744800,3,45.0
2,16,248290,1,0.0
3,18,248288,1,0.0
4,19,744903,3,0.0
5,20,248296,1,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,50,16.0
1,15,3,45.0
2,16,1,0.0
3,18,1,0.0
4,19,3,0.0
5,20,1,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,50.0
1,15,Released,3.0
2,16,Released,1.0
3,18,Released,1.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Executing,16.0,45.0,0.0,0.0,0.0,0.0
Released,50.0,3.0,1.0,1.0,3.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Executing,16.0,45.0,0.0,0.0,0.0,0.0
1,Released,50.0,3.0,1.0,1.0,3.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Executing,16.0,45.0,0.0,0.0,0.0,0.0
1,Released,50.0,3.0,1.0,1.0,3.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()